# 資料清洗
再進行資料分析之前，我先將資料進行清洗並且新增一些新的變數到資料當中

# import package

In [1]:
# data analysis
import pandas as pd
import numpy as np

# text
import re

# check date
import datetime
from datetime import date
import holidays

# web crawler
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# import data

In [2]:
# read excel
excel_file = "112年1-10月交通事故簡訊通報資料.xlsx"
df = pd.read_excel(excel_file)
# show
df.head()

,年,月,日,時,分,國道名稱,方向,里程,事件發生,交控中心\n接獲通報,...,車輛4,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局
0,2023,2,9,19,21,國道3號,南,54.0,19:21:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2023,2,10,3,36,國道1號,南,6.0,03:36:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2023,2,10,5,33,國道3號,南,76.0,05:33:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2023,2,10,7,35,國道3號,北,98.0,07:35:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2023,2,10,7,52,國道1號,北,23.0,07:52:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


## Fill in missing value of "車輛"
填補"車輛"遺失值

### define function to fill the value of "車輛"

In [3]:
# adjust "簡訊內容"
def process_message(message):
    if "追撞" in message:
        if "," in message:
            return message[message.find(")") + 1:].split(",")[0].split("追撞")[0]
        else:
            return message[message.find(")") + 1:].split("追撞")[0]
    elif "," in message:
        return message.split(",")[0]
    else:
        return message

In [4]:
# fill "車輛{i}", with adjusted "簡訊內容"
def fill_vehicles(row):
    message = row["簡訊內容"]
    # use RE to find "num + vehicle_type"(數量以及車輛類型)
    # match num & vehicle_type, not include "+"
    matches = re.findall(r'(\d+)\s*([^+]+)', message)
    # save vehicle_type
    vehicles = []
    for count, vehicle_type in matches:
        count = int(count)
        # del " "
        vehicle_type = vehicle_type.strip()
        # save times of vehicle_type
        vehicles.extend([vehicle_type] * count)
    # if no num, spilt by "+"
    if not matches:
        parts = message.split("+")
        for i, part in enumerate(parts, start=1):
            row[f"車輛{i}"] = part.strip()
    else:
        # fill
        for i, vehicle in enumerate(vehicles, start=1):
            row[f"車輛{i}"] = vehicle
    return row

In [5]:
# fill "車輛{i}" NA
# find "車輛1" is NA & "肇事車輛" != 0
filtered_df = df.loc[(df["車輛1"].isna()) & (df["肇事車輛"] != 0), ["車輛1", "肇事車輛", "簡訊內容"]]
# fill "車輛" with the "簡訊內容"
filtered_df["簡訊內容"] = filtered_df["簡訊內容"].apply(process_message)
# fill "車輛"
filtered_df = filtered_df.apply(fill_vehicles, axis=1)

# show
filtered_df


,簡訊內容,肇事車輛,車輛1,車輛2,車輛3,車輛4
476,3小客+1貨櫃車,4.0,小客,小客,小客,貨櫃車
8936,2小客車,2.0,小客車,小客車,NaN,NaN
10107,佔用內+外線4小客車,4.0,小客車,小客車,小客車,小客車
15161,佔用內線2小客車,2.0,小客車,小客車,NaN,NaN
20745,佔用內線2小客車,2.0,小客車,小客車,NaN,NaN
22945,2小車,2.0,小車,小車,NaN,NaN
22946,3小車,3.0,小車,小車,小車,NaN
22947,4小車,4.0,小車,小車,小車,小車
22948,小貨車+自小,2.0,小貨車,自小,NaN,NaN
22949,2小車,2.0,小車,小車,NaN,NaN


## refill to original data
回填到原始資料當中


In [6]:
# fill filtered_df in df with corresponding col
for index, row in filtered_df.iterrows():
    # check NA, to avoid error
    if pd.isna(df.at[index, "車輛1"]):
        # max "車輛" is 4, can change
        for i in range(1, 5):
            if f"車輛{i}" in row:
                df.at[index, f"車輛{i}"] = row[f"車輛{i}"]


## Check if it is filled in correctly
檢查是否成功填入

In [7]:
# check 
i_col = 8936
sms_content = df.iloc[i_col]['車輛2']
print(sms_content)

小客車


## sorted by time
依照時間做排序

In [8]:
# sort by time
sorted_df = df.sort_values(by=["月", "日", "時", "分"])
sorted_df

,年,月,日,時,分,國道名稱,方向,里程,事件發生,交控中心\n接獲通報,...,車輛4,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局
1747,2023,1,1,0,31,國道1號,北向,286.0,00:31:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
5090,2023,1,1,1,52,國道3號,北,42.0,01:52:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4560,2023,1,1,4,52,國道3號,北,195.3,04:07:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
5091,2023,1,1,9,2,國道2號,東,16.2,09:02:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1748,2023,1,1,9,3,國道1號,南向,367.4,09:03:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,2023,10,31,18,56,國道4號,東向,26.3,18:26:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
16274,2023,10,31,19,28,國道3號,南,55.8,19:28:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16275,2023,10,31,22,13,國道1號,南,96.0,22:13:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
12020,2023,10,31,22,34,國道3號,北向,360.4,22:34:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


## Change direction and unify standards
把方向改成統一規格 <br>
Only marked "北" or "南" <br>
(只有標示 "北"或"南") <br>




In [9]:
# adjust "方向"
sorted_df["方向"] = df["方向"].str.replace("向", "")
sorted_df["方向"] = sorted_df["方向"].str.replace("南北|雙", "雙", regex=True)
sorted_df

,年,月,日,時,分,國道名稱,方向,里程,事件發生,交控中心\n接獲通報,...,車輛4,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局
1747,2023,1,1,0,31,國道1號,北,286.0,00:31:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
5090,2023,1,1,1,52,國道3號,北,42.0,01:52:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4560,2023,1,1,4,52,國道3號,北,195.3,04:07:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
5091,2023,1,1,9,2,國道2號,東,16.2,09:02:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1748,2023,1,1,9,3,國道1號,南,367.4,09:03:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,2023,10,31,18,56,國道4號,東,26.3,18:26:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
16274,2023,10,31,19,28,國道3號,南,55.8,19:28:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16275,2023,10,31,22,13,國道1號,南,96.0,22:13:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
12020,2023,10,31,22,34,國道3號,北,360.4,22:34:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


## Delete unnecessary fields
刪除不必要的欄位

In [10]:
# del not important var
columns_to_drop = ["交控中心\n接獲通報", "CCTV\n監看現場", "CMS\n發布資訊","交控中心\n通報工務段", "事故處理\n小組出發", "事故處理\n小組抵達", "事故處理\n小組完成"]
sorted_df = sorted_df.drop(columns_to_drop, axis=1)
sorted_df

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,車輛4,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局
1747,2023,1,1,0,31,國道1號,北,286.0,00:31:00,00:39:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
5090,2023,1,1,1,52,國道3號,北,42.0,01:52:00,02:27:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4560,2023,1,1,4,52,國道3號,北,195.3,04:07:00,04:52:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
5091,2023,1,1,9,2,國道2號,東,16.2,09:02:00,09:36:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1748,2023,1,1,9,3,國道1號,南,367.4,09:03:00,09:22:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,2023,10,31,18,56,國道4號,東,26.3,18:26:00,18:56:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
16274,2023,10,31,19,28,國道3號,南,55.8,19:28:00,20:09:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16275,2023,10,31,22,13,國道1號,南,96.0,22:13:00,22:25:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
12020,2023,10,31,22,34,國道3號,北,360.4,22:34:00,23:16:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


## Corrected the column of time
The time format of some fields is incorrect <br>
Therefore, corrections are made <br>

有些欄位的時間格式錯亂 <br>
因此進行修正 <br>

---

The revised parts include : "事件發生"、"事件排除" <br>

修正的部分包含 : "事件發生"、"事件排除" <br>

In [11]:
# def fun to change type into date
def convert_to_time(x):
    if isinstance(x, datetime.time):
        return x
    else:
        # change into str, fill to len==4
        x_str = str(x).zfill(4)
        
        # del ";"
        if ";" in x_str:
            x_str = x_str.replace(";", "")
        
        # check len==4, else return none
        if len(x_str) != 4 or not x_str.isdigit():
            return None
        
        # change into date
        hour = int(x_str[:2])
        minute = int(x_str[2:])
        
        # check time range, else return none
        if hour < 0 or hour > 23 or minute < 0 or minute > 59:
            return None
        
        # change type into date
        return datetime.time(hour, minute)

In [12]:
# change "事件發生"
sorted_df["事件發生"] = sorted_df["事件發生"].apply(convert_to_time)
# change "事件排除"
sorted_df["事件排除"] = sorted_df["事件排除"].apply(convert_to_time)

# check "事件發生"
non_time_values = sorted_df[sorted_df["事件發生"].apply(lambda x: type(x) != datetime.time)]["事件發生"]
print((non_time_values))
# check "事件排除"
non_time_values = sorted_df[sorted_df["事件排除"].apply(lambda x: type(x) != datetime.time)]["事件排除"]
print((non_time_values))

# show
sorted_df

Series([], Name: 事件發生, dtype: object)
Series([], Name: 事件排除, dtype: object)


,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,車輛4,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局
1747,2023,1,1,0,31,國道1號,北,286.0,00:31:00,00:39:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
5090,2023,1,1,1,52,國道3號,北,42.0,01:52:00,02:27:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4560,2023,1,1,4,52,國道3號,北,195.3,04:07:00,04:52:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
5091,2023,1,1,9,2,國道2號,東,16.2,09:02:00,09:36:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1748,2023,1,1,9,3,國道1號,南,367.4,09:03:00,09:22:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,2023,10,31,18,56,國道4號,東,26.3,18:26:00,18:56:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
16274,2023,10,31,19,28,國道3號,南,55.8,19:28:00,20:09:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16275,2023,10,31,22,13,國道1號,南,96.0,22:13:00,22:25:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
12020,2023,10,31,22,34,國道3號,北,360.4,22:34:00,23:16:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


## Fixed the error in the column of mileage
修正里程數欄位錯誤

In [13]:
# correct "里程"
## only 1 place is wrong, change 1715.0 to 4.1
sorted_df.loc[(sorted_df['國道名稱'] == '國道5號') & (sorted_df['里程'] == 1715.0), '里程'] = 4.1

# check
sorted_df[sorted_df['國道名稱'] == '國道5號']

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,車輛4,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局
14455,2023,1,1,13,28,國道5號,南,15.6,13:28:00,14:21:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
19661,2023,1,1,13,28,國道5號,南,15.6,13:28:00,14:21:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
19662,2023,1,1,13,48,國道5號,南,15.6,13:48:00,13:55:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
19663,2023,1,1,16,51,國道5號,北,24.3,16:51:00,16:55:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
19664,2023,1,2,1,32,國道5號,南,48.7,01:32:00,02:16:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16186,2023,10,27,18,11,國道5號,南,14.8,18:11:00,18:20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
16192,2023,10,28,8,35,國道5號,北,36.4,08:35:00,09:20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
16191,2023,10,28,15,10,國道5號,北,6.4,15:10:00,15:49:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
16190,2023,10,28,16,49,國道5號,北,20.0,16:49:00,16:51:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


## Take out the "回堵長度"
取出回堵長度

In [14]:
# def fun to take out "回堵長度"
def calculate_traffic_jam_length(text):
    if "回堵" in text:
        # use RE to match
        match = re.search(r'回堵(\d+)\s*(?:K|公里|k)', text)
        if match:
            return int(match.group(1))
    return 0


In [15]:
# add new col
sorted_df["回堵長度"] = sorted_df["簡訊內容"].apply(calculate_traffic_jam_length)
sorted_df

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局,回堵長度
1747,2023,1,1,0,31,國道1號,北,286.0,00:31:00,00:39:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
5090,2023,1,1,1,52,國道3號,北,42.0,01:52:00,02:27:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4560,2023,1,1,4,52,國道3號,北,195.3,04:07:00,04:52:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
5091,2023,1,1,9,2,國道2號,東,16.2,09:02:00,09:36:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1748,2023,1,1,9,3,國道1號,南,367.4,09:03:00,09:22:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,2023,10,31,18,56,國道4號,東,26.3,18:26:00,18:56:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
16274,2023,10,31,19,28,國道3號,南,55.8,19:28:00,20:09:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
16275,2023,10,31,22,13,國道1號,南,96.0,22:13:00,22:25:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
12020,2023,10,31,22,34,國道3號,北,360.4,22:34:00,23:16:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0


In [16]:
# check some row
subset_df = sorted_df[sorted_df["回堵長度"] != 0][["簡訊內容", "回堵長度"]]
subset_df

,簡訊內容,回堵長度
5092,"北控通報3級01/01,09:39國1南向88K(結報)佔用內+中線3小客車追撞事故,無人傷...",4
5094,"北控通報3級01/01,11:15國1高架南向70K(結報)佔用內線4小客車追撞事故,無人傷...",3
14455,"坪控通報3升2級 01/01,13:28國5南向15.6K(結報)2小自追撞佔用內線事故,無...",2
19661,"坪控通報3升2級 01/01,13:28國5南向15.6K(結報)2小自追撞佔用內線事故,無...",2
5099,"北控通報3級01/01,15:46國1北向87.6K(結報)佔用內線3小客車追撞事故,無人傷...",4
...,...,...
16272,"北控通報3級10/31,15:19國1北向五股出口(結報)2小車追撞佔用外線事故,無人受傷,...",1
493,"中控通報3級10/31,15:59國3南向157.9K(結報)1小客追撞施工緩撞車事故占內2...",3
491,"中控通報3級10/31,17:43國3北向166.5K(結報)3小客車追撞事故占中間車道,1...",1
16273,"北控通報3級10/31,18:33國2東向4.8K(結報)2小車追撞佔用內線事故,無人受傷,...",3


## correct the column of "國道名稱"
修正 "國道名稱"

In [17]:
# change "國3甲" to "國道3甲", "國2甲" to "國道2甲"
sorted_df.loc[sorted_df['國道名稱'] == "國3甲", '國道名稱'] = "國道3甲"
sorted_df.loc[sorted_df['國道名稱'] == "國2甲", '國道名稱'] = "國道2甲"

## Delete the missing columns of "國道名稱"
刪除 "國道名稱" 有遺失的橫列

In [18]:
# del NA
sorted_df.dropna(subset=['國道名稱'], inplace=True)

In [19]:
# check
na_values = sorted_df[sorted_df['國道名稱'].isna()]
na_values

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局,回堵長度


## Keep one of the same rows
把相同的橫列留下其一

In [20]:
# unique
sorted_df = sorted_df.drop_duplicates(subset=['年', '月', '日', '時', '分', '國道名稱', '方向', '里程'])
# check
sorted_df

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局,回堵長度
1747,2023,1,1,0,31,國道1號,北,286.0,00:31:00,00:39:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
5090,2023,1,1,1,52,國道3號,北,42.0,01:52:00,02:27:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4560,2023,1,1,4,52,國道3號,北,195.3,04:07:00,04:52:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
5091,2023,1,1,9,2,國道2號,東,16.2,09:02:00,09:36:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1748,2023,1,1,9,3,國道1號,南,367.4,09:03:00,09:22:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,2023,10,31,18,56,國道4號,東,26.3,18:26:00,18:56:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
16274,2023,10,31,19,28,國道3號,南,55.8,19:28:00,20:09:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
16275,2023,10,31,22,13,國道1號,南,96.0,22:13:00,22:25:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
12020,2023,10,31,22,34,國道3號,北,360.4,22:34:00,23:16:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0


 ## Modify the colume of "車輛"
 修改車輛欄位

In [21]:
# use copy to avoid error
sorted_df_copy = sorted_df.copy()
# all columns to adjust
vehicle_columns = ["車輛1", "車輛2", "車輛3", "車輛4", "車輛5",
                   "車輛6", "車輛7", "車輛8", "車輛9", "車輛10", "車輛11", "車輛12"]

# replace values
replace_dict = {
    "重型機車": "重機",
    "小自客\n": "小自客", "小客車": "小自客", "小客": "小自客", "小車": "小自客", '小自客\"\n': "小自客", " 小自客": "小自客", "自小": "小自客", "小自車": "小自客", "小自客_x000D_\n": "小自客",
    "大客車": "大客車", "大客車\n": "大客車", "大客": "大客車", "大客車_x000D_\n": "大客車",
    "貨車": "貨車",
    "小貨車": "小貨車", "小貨車\n": "小貨車", "小自貨": "小貨車", "小營業貨": "小貨車", "小貨": "小貨車", "小貨車_x000D_\n": "小貨車",
    "中型貨車": "中貨車",
    "大貨車": "大貨車", "貨櫃車": "大貨車", " 大貨車": "大貨車", "大貨車\n": "大貨車", "大貨": "大貨車", "大貨車ˉ": "大貨車", "284貨車": "大貨車", "大貨\n": "大貨車", "大貨車_x000D_\n": "大貨車",
    "聯結車": "聯結車", "聯結車\n": "聯結車",
    "拖吊車": "拖吊車",
    "槽車": "槽車",
    "垃圾車": "垃圾車",
    "貨櫃車": "貨櫃車",
    "工工程車": "工程車", "工程車\n": "工程車",
    "施工車": "施工車",
    "緩撞車": "緩撞車", "緩撞車\n": "緩撞車", "施工緩撞車": "緩撞車", "\"事故班緩撞車": "緩撞車", "事故班緩撞車": "緩撞車",
    "油罐車": "油罐車",
    "預拌水泥車": "預拌車", "預拌混泥土車": "預拌車",
    "砂石車": "砂石車",
    "公警": "公警",
    "拖板車": "拖板車",
    "計程車": "計程車",
    "起重機": "起重機",
    "其他": "其他",
    "國光客運": "中型巴士",
    "施工高空車": "高空作業車",
    "     ": np.nan
}

# adjust and replace values in columns
for column in vehicle_columns:
    sorted_df_copy.loc[:, column] = sorted_df_copy.loc[:, column].replace(replace_dict)
    sorted_df = sorted_df_copy

    # show the adjusted results
    counts = sorted_df[column].value_counts()
    print(f"{column} 調整後的結果：")
    print(counts)
    print("-" * 50)


車輛1 調整後的結果：
小自客    15831
小貨車      923
大貨車      665
大客車      133
其他        87
聯結車       56
緩撞車        6
貨櫃車        5
貨車         5
拖吊車        5
計程車        4
施工車        4
槽車         4
中貨車        2
垃圾車        2
起重機        2
工程車        1
油罐車        1
連結車        1
砂石車        1
預拌車        1
拖板車        1
公警         1
Name: 車輛1, dtype: int64
--------------------------------------------------
車輛2 調整後的結果：
小自客       13476
小貨車        1227
大貨車         836
其他          254
大客車         117
聯結車          29
緩撞車          29
計程車          25
拖吊車           5
施工車           3
砂石車           2
重機            2
槽車            2
高空作業車         2
工程車           2
垃圾車           2
機車            2
中型巴士          2
小客貨           1
貨車            1
施工標緻車         1
中客車           1
救護車           1
貨櫃車           1
中貨車           1
施工掃地車         1
事故班標誌車        1
板車            1
警戒車           1
油罐車           1
水泥車           1
Name: 車輛2, dtype: int64
--------------------------------------------------
車輛3 調整後的結果：
小自客    2506
小貨車    

### Check if there are missing values
檢查是否存在遺失值

In [22]:
# fill "車輛{i}" NA
# find "車輛1" is NA & "肇事車輛" != 0
filtered_df = df.loc[(df["車輛1"].isna()) & (df["肇事車輛"] != 0), ["車輛1", "肇事車輛", "簡訊內容"]]

# adjust "簡訊內容"
def process_message(message):
    if "追撞" in message:
        if "," in message:
            return message[message.find(")") + 1:].split(",")[0].split("追撞")[0]
        else:
            return message[message.find(")") + 1:].split("追撞")[0]
    elif "," in message:
        return message.split(",")[0]
    else:
        return message

filtered_df["簡訊內容"] = filtered_df["簡訊內容"].apply(process_message)

# show
filtered_df

,車輛1,肇事車輛,簡訊內容


## See what variables are currently left
查看目前剩下哪些變數

In [23]:
# check saved cols
column_names = sorted_df.columns.tolist()
print(column_names)

['年', '月', '日', '時', '分', '國道名稱', '方向', '里程', '事件發生', '事件排除', '處理分鐘', '事故類型', '死亡', '受傷', '內路肩', '內車道', '中內車道', '中車道', '中外車道', '外車道', '外路肩', '匝道', '簡訊內容', '翻覆事故\n註記', '施工事故\n註記', '危險物品車輛\n註記', '車輛起火\n註記', '冒煙車事故\n註記', '主線中斷\n註記', '肇事車輛', '車輛1', '車輛2', '車輛3', '車輛4', '車輛5', '車輛6', '車輛7', '車輛8', '車輛9', '車輛10', '車輛11', '車輛12', '分局', '回堵長度']


## Added new variable "是否為假日"
新增變數 "是否為假日" <br>

"假日" include Saturday, Sunday, and festivals <br>

假日包含 六、日、節日 <br>

### Define the function is_holiday to determine holidays
定義函數 is_holiday 判斷假日

In [24]:
# set country--TW is in CN
cn_holidays = holidays.CountryHoliday('CN')

# def func to know is holiday/weekend or weekday
def is_holiday(check_date):
    # check whether in holiday
    if check_date in cn_holidays:
        return True
    # check whether in weekend
    ## 6,7 is day num
    if check_date.isoweekday() in [6, 7]:
        return True
    return False

### Add variables
加入變數

In [25]:
# add new var --"是否為假日"
sorted_df['是否為假日'] = sorted_df.apply(
    lambda row: 1 if is_holiday(date(row['年'], row['月'], row['日'])) else 0, axis=1)

In [26]:
# show
sorted_df

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局,回堵長度,是否為假日
1747,2023,1,1,0,31,國道1號,北,286.0,00:31:00,00:39:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,1
5090,2023,1,1,1,52,國道3號,北,42.0,01:52:00,02:27:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1
4560,2023,1,1,4,52,國道3號,北,195.3,04:07:00,04:52:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,1
5091,2023,1,1,9,2,國道2號,東,16.2,09:02:00,09:36:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1
1748,2023,1,1,9,3,國道1號,南,367.4,09:03:00,09:22:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,2023,10,31,18,56,國道4號,東,26.3,18:26:00,18:56:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0
16274,2023,10,31,19,28,國道3號,南,55.8,19:28:00,20:09:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
16275,2023,10,31,22,13,國道1號,南,96.0,22:13:00,22:25:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0
12020,2023,10,31,22,34,國道3號,北,360.4,22:34:00,23:16:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,0


## Add a new variable "相近時間是否有事件"
新增變數 "相近時間是否有事件"

This variable will be set to 1 only if both similar time and similar location are met <br>
Definition of close time: B occurs at the time of A's occurrence + processing time $\rightarrow$ B is True here <br>
Definition of similar location: The location (mileage) of B is within 10km of the location (mileage) of A. <br>


同時符合相近時間&相近地點才會將此變數設為 1 <br>
相近時間定義 : B 的發生時間在 A 的發生時間+處理時間 $\rightarrow$ B 在此為 True <br>
相近地點定義 : B 的發生地點(里程) 與 A 的發生地點(里程) 相差 10km 之內 <br>

### Add variables
加入變數

In [27]:
# add new var --"是否為相近時間"、"是否為相近地點"、"相近時間是否有事件"
sorted_df['是否為相近時間'] = 0
sorted_df['是否為相近地點'] = 0
sorted_df['相近時間&地點是否有事件'] = 0
# first--group to process
grouped = sorted_df.groupby(['年', '月', '日', '國道名稱', '方向'])

In [ ]:
# loop
for name, group in grouped:
    # repeat every group
    for i in range(len(group)):
        current_event = group.iloc[i]
        current_start = current_event['事件發生']
        current_end = current_event['事件排除']
        current_mileage = current_event['里程']
        
        # check the logic
        for j in range(i):
            previous_event = group.iloc[j]
            previous_start = previous_event['事件發生']
            previous_end = previous_event['事件排除']
            previous_mileage = previous_event['里程']
            
            # check place
            if abs(current_mileage - previous_mileage) <= 10:
                sorted_df.at[current_event.name, '是否為相近地點'] = 1
            # check time
            if (previous_start <= current_start <= previous_end):
                sorted_df.at[current_event.name, '是否為相近時間'] = 1
            # check both
            if abs(current_mileage - previous_mileage) <= 10:
                if (previous_start <= current_start <= previous_end):
                    sorted_df.at[current_event.name, '相近時間&地點是否有事件'] = 1
                    break
# # cross 2 var
# sorted_df['相近時間是否有事件'] = sorted_df['是否為相近時間'] * sorted_df['是否為相近地點']
print("finish")
# using 12.4s

finish


### Check if the variables are correct
檢查變數是否正確判斷

In [29]:
# id connect to mdndhm.xlsx
# id +2 == xlsx id
new_df = sorted_df.sort_values(
    by=["年", "月", "日", "國道名稱", "方向", "時", "分"]).reset_index(drop=True)
# filtered_df = new_df[new_df["是否為相近地點"] == 1]
# filtered_df = new_df[new_df["是否為相近時間"] == 1]
filtered_df = new_df[new_df["相近時間&地點是否有事件"] == 1]
# filtered_df = new_df[(new_df["是否為相近地點"] == 1) & (new_df["是否為相近時間"] == 0)]
filtered_df

# 8318/17742 hace accident in near place with mile set == 10
# 3705/17742 have accident in near time
# 837/17742 have accident in near time and near place

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,車輛9,車輛10,車輛11,車輛12,分局,回堵長度,是否為假日,是否為相近時間,是否為相近地點,相近時間&地點是否有事件
40,2023,1,1,18,58,國道3號,北,230.8,18:35:00,18:58:00,...,NaN,NaN,NaN,NaN,2,2,1,1,1,1
58,2023,1,1,13,48,國道5號,南,15.6,13:48:00,13:55:00,...,NaN,NaN,NaN,NaN,4,0,1,1,1,1
75,2023,1,2,17,33,國道1號,北,35.7,17:33:00,18:01:00,...,NaN,NaN,NaN,NaN,1,4,1,1,1,1
81,2023,1,2,18,37,國道1號,北,121.0,17:55:00,18:37:00,...,NaN,NaN,NaN,NaN,2,3,1,1,1,1
82,2023,1,2,18,39,國道1號,北,355.1,18:39:00,18:42:00,...,NaN,NaN,NaN,NaN,3,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17610,2023,10,28,16,20,國道1號,南,210.0,16:01:00,16:20:00,...,NaN,NaN,NaN,NaN,2,3,1,1,1,1
17611,2023,10,28,16,23,國道1號,南,210.5,16:02:00,16:23:00,...,NaN,NaN,NaN,NaN,2,3,1,1,1,1
17630,2023,10,28,11,44,國道3號,南,58.3,11:44:00,11:47:00,...,NaN,NaN,NaN,NaN,1,4,1,1,1,1
17632,2023,10,28,12,2,國道3號,南,55.8,12:02:00,12:08:00,...,NaN,NaN,NaN,NaN,1,3,1,1,1,1


## Add variables "事故車是否包含非小客車"
新增變數 "事故車是否包含非小客車"

In [30]:
# def func to know every "車輛", without NA
def check_vehicle_type(row):
    # set ori 0
    vehicle_status = 0
    vehicle_columns = [col for col in sorted_df.columns if col.startswith('車輛')]
    
    # check every "車輛{i}" col
    for col in vehicle_columns:
        # check not NA
        if pd.notna(row[col]):
            # check "小客車"
            if "小自客" in str(row[col]):
                vehicle_status = 0
            else:
                vehicle_status = 1
    return vehicle_status

# add new col"事故車是否包含非小客車"
sorted_df['事故車是否包含非小客車'] = sorted_df.apply(check_vehicle_type, axis=1)
# check
sorted_df[['事故車是否包含非小客車'] + vehicle_columns + ["簡訊內容"]]

,事故車是否包含非小客車,車輛1,車輛2,車輛3,車輛4,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,簡訊內容
1747,0,小自客,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"南控通報3級01/01,00:31國1北向286k(結報)小客車撞護欄,00:39事故排除,..."
5090,0,小自客,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"北控通報3級01/01,01:52國3北向土城入口(結報)1小客自撞事故,無傷亡,無回堵,0..."
4560,0,小自客,小自客,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"中控通報3級01/01,04:07國3北向195.3K(結報)2小客車追撞事故占內2車道,0..."
5091,0,小自客,小自客,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"北控通報3級01/01,09:02國2東向16.2K(結報)外路肩(更正)2小客車追撞事故,..."
1748,0,小自客,小自客,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"南控通報3級01/01,09:03國1南向高雄中正路入口(結報)2小客車追撞,09:22事故..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,0,小自客,小自客,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"中控通報3級10/31,18:26國4東向26.3K(結報)2小客車追撞事故占內車道,18:..."
16274,0,小自客,小自客,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"北控通報3級10/31,19:28國3南向55.8K(結報)2小自追撞佔用內線車道事故,無人..."
16275,0,小自客,小自客,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"北控通報3級10/31,22:13國1南向96K(結報)2小自追撞佔用中內線車道事故,無人受..."
12020,1,小自客,其他,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"南控通報2級10/31,22:34國3北向360.4k(結報修正)小客車撞鹿,23:16事故..."


In [31]:
# get "事故車是否包含非小客車" counts
value_counts = sorted_df['事故車是否包含非小客車'].value_counts()
value_counts

0    14419
1     3323
Name: 事故車是否包含非小客車, dtype: int64

## See what variables are currently left
查看目前剩下哪些變數

In [32]:
# check saved cols
column_names = sorted_df.columns.tolist()
print(column_names)

['年', '月', '日', '時', '分', '國道名稱', '方向', '里程', '事件發生', '事件排除', '處理分鐘', '事故類型', '死亡', '受傷', '內路肩', '內車道', '中內車道', '中車道', '中外車道', '外車道', '外路肩', '匝道', '簡訊內容', '翻覆事故\n註記', '施工事故\n註記', '危險物品車輛\n註記', '車輛起火\n註記', '冒煙車事故\n註記', '主線中斷\n註記', '肇事車輛', '車輛1', '車輛2', '車輛3', '車輛4', '車輛5', '車輛6', '車輛7', '車輛8', '車輛9', '車輛10', '車輛11', '車輛12', '分局', '回堵長度', '是否為假日', '是否為相近時間', '是否為相近地點', '相近時間&地點是否有事件', '事故車是否包含非小客車']


## Add variables "距離交流道多遠" (not done yet)
新增變數 "距離交流道多遠"

### 載入"國道一號"的交流道表格

In [ ]:
# webdriver
driver_path = "D:/geckodriver/chromedriver.exe"
service = Service(driver_path)
# open driver
driver = webdriver.Chrome(service=service)
# page url
url = "https://www.freeway.gov.tw/Publish.aspx?cnid=1906&p=4617"
driver.get(url)

# turn into table's XPATH
xpath = "/html/body/form/div[4]/div/div/div[2]/div[2]/div/div/table[1]/tbody"
table_element = driver.find_element(By.XPATH, xpath)

# get every col
rows = table_element.find_elements(By.TAG_NAME, "tr")
# save in list
table_data = []
# get every val in df
for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    cols = [col.text for col in cols]
    table_data.append(cols)
# quit
driver.quit()
# change to df
aaa = pd.DataFrame(table_data)

In [ ]:
# 保留第2列是整數的行，並忽略空值
aaa = aaa[aaa.iloc[:, 1].apply(lambda x: str(x).isdigit())]
aaa

In [ ]:
aaa.to_csv("table_data_numeric_second_column.csv", index=False, encoding="utf-8-sig")

## Save the currently adjusted data set
儲存目前已經調整過的資料集

In [34]:
# save
sorted_df.to_excel('seven_edit_df.xlsx', index=False)

### check the data

In [35]:
# read excel
excel_file = "seven_edit_df.xlsx"
sorted_df = pd.read_excel(excel_file)
# show
sorted_df.head()

,年,月,日,時,分,國道名稱,方向,里程,事件發生,事件排除,...,車輛10,車輛11,車輛12,分局,回堵長度,是否為假日,是否為相近時間,是否為相近地點,相近時間&地點是否有事件,事故車是否包含非小客車
0,2023,1,1,0,31,國道1號,北,286.0,00:31:00,00:39:00,...,NaN,NaN,NaN,3,0,1,0,0,0,0
1,2023,1,1,1,52,國道3號,北,42.0,01:52:00,02:27:00,...,NaN,NaN,NaN,1,0,1,0,0,0,0
2,2023,1,1,4,52,國道3號,北,195.3,04:07:00,04:52:00,...,NaN,NaN,NaN,2,0,1,0,0,0,0
3,2023,1,1,9,2,國道2號,東,16.2,09:02:00,09:36:00,...,NaN,NaN,NaN,1,0,1,0,0,0,0
4,2023,1,1,9,3,國道1號,南,367.4,09:03:00,09:22:00,...,NaN,NaN,NaN,3,0,1,0,0,0,0
